In [1]:
from elasticsearch import Elasticsearch
import os
import py_vncorenlp
import json
import re
from tqdm import tqdm, trange
import requests

In [2]:
# vncorenlp_segmentor = py_vncorenlp.VnCoreNLP(max_heap_size='-Xmx8g', annotators=['wseg'], save_dir='/workspace/nlplab/kienvt/scada-tokenize-server/vncorenlp')

2024-04-27 15:15:47 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch

# tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2', cache_dir ='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache')
# model = AutoModel.from_pretrained('vinai/phobert-base-v2', cache_dir ='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache').to('cuda:0')

# model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder', 
#                             cache_folder='./hf_cache').to('cuda:0')

/workspace/nlplab/kienvt/KLTN/kenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer('kien-vu-uet/finetune-vietnamese-bi-encoder', 
                            cache_folder='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache').to('cuda:0')

In [5]:
# from sentence_transformers import SentenceTransformer, models, evaluation
# word_embedding_model = models.Transformer('bigscience-catalogue-lm-data/sgpt-nli-bloom-1b3', max_seq_length=256, cache_dir='./hf_cache')
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# model2 = SentenceTransformer(modules=[word_embedding_model, pooling_model]).to('cuda:0')

In [7]:
# model2.encode('Tôi là học sinh trường Đại học Công Nghệ').shape

(2048,)

In [4]:
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY', 'Ylh4SkRJOEJTcEJhOGhwYlY4TFo6R0p3NUJ6SHFUVTZJd01FSm56RTRtdw==')

In [5]:
client = Elasticsearch('http://localhost:9200', api_key=ELASTIC_API_KEY)

In [6]:
client.info()

ObjectApiResponse({'name': 'elasticsearch', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BQnO6xnOSha1XK5s1S0PfA', 'version': {'number': '8.12.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6185ba65d27469afabc9bc951cded6c17c21e3f3', 'build_date': '2024-02-01T13:07:13.727175297Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
def create_index(name, b=0.75, k=1.2, dims=768):
    settings = {
        "number_of_shards": 1,
        "index" : {
            "similarity" : {
                "default" : {
                    "type" : "BM25",
                    "b": b,
                    "k1": k,
                }
            }
        }
    }

    mappings = {
        "properties": {
            "content_vector": {
                "type": "dense_vector",
                "dims": dims,
                "index": "true",
                "similarity": "cosine",
            }
        }
    }

    client.indices.delete(index=name, ignore_unavailable=True)
    client.indices.create(index=name, settings=settings, mappings=mappings)


In [8]:
def segment(text, sent_separator=' '):
    # _text = re.sub('\.(\s)?\.', '.', text)
    # _text = _text.split('.')
    # i = 0
    # while i + 1 < len(_text):
    #     _text[i] = _text[i].strip()
    #     if len(_text[i]) <= 7: 
    #         _text[i+1] = f'{_text[i]} {_text[i+1].strip()}'
    #         _text.pop(i)
    #     else:
    #         i += 1
    # _text = [_t for _t in _text if len(_t) > 0]
    # # print(_text)
    # # print([vncorenlp_segmentor.word_segment(_t) for _t in _text])
    # return '. '.join([' '.join(vncorenlp_segmentor.word_segment(_t)) for _t in _text])
    response = requests.post('http://localhost:9091/segment2', json={'text': text, 'sent_separator': sent_separator})
    return response.json().get('sent')

In [9]:
segment(
    """2.. Bãi bỏ Thông tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ trưởng Bộ Giao thông vận tải quy định về biểu mẫu, giấy chứng nhận và số kiểm tra an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa. """ \
    """3. Các giấy chứng nhận, số an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa được cấp trước ngày 01 tháng 9 năm 2017 theo Thông tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ trưởng Bộ Giao thông vận tải, """ \
    """quy định về biểu mẫu giấy chứng nhận và số kiểm tra an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa sẽ tiếp tục có hiệu lực đến ngày hết hiệu lực của các giấy chứng nhận và số này."""
)

'2 .. Bãi_bỏ Thông_tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ_trưởng Bộ Giao_thông vận_tải quy_định về biểu_mẫu , giấy chứng_nhận và số kiểm_tra an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa . 3 . Các giấy chứng_nhận , số an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa được cấp trước ngày 01 tháng 9 năm 2017 theo Thông_tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ_trưởng Bộ Giao_thông vận_tải , quy_định về biểu_mẫu giấy chứng_nhận và số kiểm_tra an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa sẽ tiếp_tục có hiệu_lực đến ngày hết hiệu_lực của các giấy chứng_nhận và số này .'

In [10]:
def prepare_content(header, content) -> str:
    sent_separator='<\\>'
    segment_headers = segment(header, sent_separator).split(sent_separator)
    segment_contents = segment(content, sent_separator).split(sent_separator)
    segment_headers.reverse()
    for _head in segment_headers:
        if _head not in segment_contents:
            segment_contents.insert(0, _head)
    return ' '.join(segment_contents)

In [11]:
content = """Các mối hàn hoàn chỉnh phải thoả mãn người giám sát . Khe hở giữa các bề mặt được vát của các phần tử được nối với nhau phải được giữ ở mức nhỏ nhất . Khi khe hở giữa các phần tử được nối với nhau vượt quá 2,0 mm và không quá 5 mm , kích thước chân mối hàn phải được tăng lên bằng khe hở . Khi khe hở giữa các phần tử lớn hơn 5 mm , quy trình sửa chữa phải được người giám sát chấp thuận . 43 6.3.7 Kiểm tra không phá huỷ ( NDT ) 6.3.7.1 Yêu cầu chung 6.3.7.1.1 Trước khi bắt đầu bất kỳ đợt NDT nào , kế hoạch kiểm tra NDT phải được trình cho người giám sát xem xét và chấp thuận , và phải tuân theo 6.2.6.2.1 - ( 3 ) của TCVN 12823-5 : 2020 và 6.3.6.3 của Tiêu chuẩn này . 6.3.7.1.2 Toàn bộ quy trình NDT phải được xem xét và chấp nhận bởi người giám sát trước khi thực hiện kiểm tra NDT . Kiểm tra bằng chụp phim ( RT ) , kiểm tra bằng siêu âm ( UT ) , kiểm tra bằng hạt từ tính ( MPI ) , kiểm tra bằng phương pháp thẩm thấu ( PT ) , dòng điện xoáy ( EC ) hoặc đo trường dòng điện xoay chiều ( ACFM ) phải được tiến hành thoả mãn người giám sát . Ngoại trừ kiểm tra bằng chụp phim ( RT ) , người giám sát có thể yêu cầu chứng kiến NDT được thực hiện bởi người được chứng nhận ."""
header = """Ngoại trừ kiểm tra bằng chụp phim ( RT ) , người giám sát có thể yêu cầu chứng kiến NDT được thực hiện bởi người được chứng nhận . [1]H, [2], [3], [4], [5], [6], [1]M, [7], [8] Chương trình kiểm soát chế tạo thân giàn (HCMP) cho các khu vực kết cấu nguy hiểm được chuẩn bị bởi nhà máy chế tạo và được trình nộp để thẩm định trước khi bắt đầu chế tạo thân giàn, chương trình này phải bao gồm các dữ liệu sau đây: a) Kết cấu đặc biệt, thường sử dụng cho các khu vực kết cấu nguy hiểm nhất; b) Kết cấu chính, thường sử dụng cho các khu vực kết cấu nguy hiểm; c) Kết cấu phụ, thường áp dụng cho các kết cấu ít nguy hiểm nhất."""

prepare_content(header, content)

'[ 1 ] H , [ 2 ] , [ 3 ] , [ 4 ] , [ 5 ] , [ 6 ] , [ 1 ] M , [ 7 ] , [ 8 ] Chương_trình kiểm_soát chế_tạo thân giàn ( HCMP ) cho các khu_vực kết_cấu nguy_hiểm được chuẩn_bị bởi nhà_máy chế_tạo và được trình nộp để thẩm_định trước khi bắt_đầu chế_tạo thân giàn , chương_trình này phải bao_gồm các dữ_liệu sau đây : a ) Kết_cấu đặc_biệt , thường sử_dụng cho các khu_vực kết_cấu nguy_hiểm nhất ; b ) Kết_cấu chính , thường sử_dụng cho các khu_vực kết_cấu nguy_hiểm ; c ) Kết_cấu phụ , thường áp_dụng cho các kết_cấu ít nguy_hiểm nhất . Các mối hàn hoàn_chỉnh phải thoả_mãn người giám_sát . Khe hở giữa các bề_mặt được vát của các phần_tử được nối với nhau phải được giữ ở mức nhỏ nhất . Khi khe hở giữa các phần_tử được nối với nhau vượt quá 2,0 mm và không quá 5 mm , kích_thước chân mối hàn phải được tăng lên bằng khe hở . Khi khe hở giữa các phần_tử lớn hơn 5 mm , quy_trình sửa_chữa phải được người giám_sát chấp_thuận . 43 6.3.7 Kiểm_tra không phá_huỷ ( NDT ) 6.3.7.1 Yêu_cầu chung 6.3.7.1.1 Trước

In [12]:
def push_data_to_elasticsearch(index_name, path, encoder, proc_bsz=100):
    # path = './chunks-7'
    operations = []
    errors = []
    for filename in os.listdir(path):
        fp = os.path.join(path, filename)
        lines = open(fp, 'r').readlines()
        last_symbol_no = ''
        pbar = tqdm(lines, desc='Process ' + filename)
        for line in pbar:
            try:
                chunk = json.loads(line)
                item = {
                    "page_url" : chunk["page_url"],
                    "category" : chunk["category"],
                    # "attachment_url" : chunk["attachment_url"],
                    "title": chunk["title"],
                    "symbol_number": chunk["symbol_number"],
                    "field": chunk["field"],
                    "description": chunk["description"],
                    "content" : chunk["content"],
                    "header": chunk["header"],
                    "chunk_index": chunk["chunk_index"]
                }
                # print(json.dumps(item, indent=4, ensure_ascii=False))
                item['content_vector'] = encoder.encode(segment(prepare_content(item["header"], item["content"]))).tolist()
                operations.append({"index": {"_index": index_name}})
                operations.append(item)
                last_symbol_no = chunk["symbol_number"]
            except Exception as e:
                errors.append((e.args, fp, lines.index(line)))
            pbar.set_postfix({"errors": len(errors), "symbol": f"{last_symbol_no}#{chunk['chunk_index']}"})

    assert len(operations) % 2 == 0
    print('Number of documents:', len(operations) // 2)
    
    batch_operations = []
    for i in tqdm(range(0, len(operations), 2), desc='Push to ' + str(index_name)):
        batch_operations.append(operations[i])
        batch_operations.append(operations[i+1])
        if len(batch_operations) == proc_bsz:
            client.bulk(index=index_name, operations=batch_operations, refresh=True)
            batch_operations.clear()

    if len(batch_operations) > 0:
        client.bulk(index=index_name, operations=batch_operations, refresh=True)
        batch_operations.clear()
        
    return errors

In [13]:
for overlapse in [30]:
    print('Overlapse', overlapse)
    index_name = f"vnr-overlapse{overlapse:02d}-finetuned-biencoder"
    data_path = f"/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-7/overlapse_{overlapse}"

    create_index(index_name)

    errors = push_data_to_elasticsearch(index_name, path=data_path, encoder=model, proc_bsz=100)
    print(errors)
    print('======')

Overlapse 10


Process proc-3.txt: 100%|██████████| 6587/6587 [21:58<00:00,  4.99it/s, errors=0, symbol=Số ký hiệu: QCVN 55:2013/BGTVT#70]               


Number of documents: 35696


Push to vnr-overlapse10-finetuned-biencoder: 100%|██████████| 35696/35696 [02:24<00:00, 246.27it/s]


[]
Overlapse 30


Process proc-2.txt:  48%|████▊     | 5365/11175 [18:09<19:40,  4.92it/s, errors=0, symbol=Số ký hiệu: QCVN 54: 2019/BGTVT#119] 


KeyboardInterrupt: 

---

In [ ]:
# pip install -q transformers accelerate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel

checkpoint = "bigscience/mt0-large"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto", 
                                              cache_dir='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache')



In [ ]:
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
outputs = model(input_ids=inputs)
# print(tokenizer.decode(outputs[0]))
outputs